In [3]:
card_pool_names = [
    "Skinshifter",
"Pack Guardian",
"Supreme Phantom",
"Ninja of the Deep Hours",
"Negate",
"Familiar's Ruse",
"Thought Courier",
"Riddleform",
"Grazing Gladehart",
"Filigree Familiar",
"Prey Upon",
"Foul Orchard",
"Llanowar Wastes",
"Sulfur Falls",
"Killing Glare",
"Wizard's Retort",
"Higure, the Still Wind",
"Rattlechains",
"Centaur's Herald",
"Harbinger of the Tides",
"Essence Flux",
"Territorial Allosaurus",
"Greenwarden of Murasa",
"Foundry of the Consuls",
"Scrounging Bandar",
"Woodland Stream",
"Kiri-Onna",
"Isolated Chapel",
"Highland Lake",
"Sidisi, Brood Tyrant",
"Naru Meha, Master Wizard",
"Ludevic's Test Subject",
"Bounding Krasis",
"Opt",
"Mana Leak",
"Simic Signet",
"Dissolve",
"Supreme Will",
"Kessig Cagebreakers",
"Birds of Paradise",
"River Hoopoe",
"Crawling Sensation",
"Temple of Mystery",
"Submerged Boneyard",
"Battlefield Forge",
]

In [4]:
from mtg_draft_ai.api import *

cards = read_cube_toml('cube_81183_tag_data.toml')

In [5]:
cards_by_name = {c.name: c for c in cards}

In [6]:
card_pool = [cards_by_name[name] for name in card_pool_names]

In [8]:
from mtg_draft_ai import synergy
import networkx as nx


_NONLANDS_IN_DECK_DEFAULT = 23

_COLOR_COMBOS = ['WU', 'WB', 'WR', 'WG', 'UB', 'UR', 'UG', 'BR', 'BG', 'RG']

def _comm_score(current_graph, comm):
    build_with_comm = list(current_graph.nodes) + list(comm)
    graph_with_comm = synergy.create_graph(build_with_comm, remove_isolated=False)
    edges_added = len(graph_with_comm.edges) - len(current_graph.edges)
    return edges_added / len(comm)


def _num_nonlands(current_build):
    return len([c for c in current_build if 'land' not in c.types])


def _communities_build(card_pool_graph, target_playables):
    """Builds a deck by trying combinations of communities. Ignores all other factors (including color).

    Outline of algorithm:

    1). Split the on-color cards into communities using networkx's greedy_modular_communities
    2). Find the "best" community to add based on # of edges it would add / # nodes
    3). Add the entire community to the pool
    4). Repeat 2-3 until we have >= the target number of playables
    5). Cut cards one by one based on lowest centrality score in the graph for the pool
    until we have the target number of playables

    Args:
        card_pool_graph (networkx.Graph): A synergy graph of Cards as the card pool to build from.
        target_playables (int): The number of nonland cards to include in the deck. Utility lands can still be
            included if they are beneficial, but don't count towards this total.

    Returns:
        List[Card]: The build for the given pool pool of cards.
    """
    if _num_nonlands(card_pool_graph.nodes) < target_playables:
        raise DeckbuildError('Not enough nonland cards')

    communities = nx.algorithms.community.greedy_modularity_communities(card_pool_graph)

    # Add the community with the best ratio of edges added to nodes added. Repeat until we have >=
    # the target number of playables.
    current_build = []
    while _num_nonlands(current_build) < target_playables:
        current_graph = synergy.create_graph(current_build, remove_isolated=False)
        communities.sort(key=lambda c: _comm_score(current_graph, c), reverse=True)
        current_build.extend(communities.pop(0))

    current_build_graph = synergy.create_graph(current_build, remove_isolated=False, freeze=False)

    # Cut least-central cards one by one until we're at the final number of playables
    while _num_nonlands(current_build_graph.nodes) > target_playables:
        centralities = synergy.sorted_centralities(current_build_graph)
        least_central_card = centralities[-1][0]
        current_build_graph.remove_node(least_central_card)

    return list(current_build_graph.nodes)

In [33]:
colors = 'UG'

graph = synergy.create_graph(card_pool, remove_isolated=False)


on_color_with_utility_lands = [c for c in card_pool
            if synergy.castable(c, colors) and not ('land' in c.types and not c.tags)]
on_color_subgraph = graph.subgraph(on_color_with_utility_lands)
#deck_for_colors = _communities_build(on_color_subgraph, 23)

target_playables = 23
card_pool_graph = on_color_subgraph


communities = nx.algorithms.community.greedy_modularity_communities(card_pool_graph)

# Add the community with the best ratio of edges added to nodes added. Repeat until we have >=
# the target number of playables.
current_build = []
while _num_nonlands(current_build) < target_playables:
    current_graph = synergy.create_graph(current_build, remove_isolated=False)
    communities.sort(key=lambda c: _comm_score(current_graph, c), reverse=True)

    comm_to_add = communities.pop(0)
    print('adding comm: {}'.format([c.name for c in comm_to_add]))
    
    current_build.extend(comm_to_add)
    
print([[card.name for card in comm] for comm in communities])

current_build_graph = synergy.create_graph(current_build, remove_isolated=False, freeze=False)

# Cut least-central cards one by one until we're at the final number of playables
while _num_nonlands(current_build_graph.nodes) > target_playables:
    centralities = synergy.sorted_centralities(current_build_graph)
    least_central_card = centralities[-1][0]
    current_build_graph.remove_node(least_central_card)

#return list(current_build_graph.nodes)

adding comm: ['Bounding Krasis', 'Dissolve', "Familiar's Ruse", 'Skinshifter', 'Mana Leak', "Wizard's Retort", 'Negate', 'Pack Guardian', "Centaur's Herald"]
adding comm: ['Rattlechains', 'Supreme Phantom', 'Higure, the Still Wind', 'Birds of Paradise', 'River Hoopoe', 'Kiri-Onna', 'Ninja of the Deep Hours']
adding comm: ['Prey Upon', 'Harbinger of the Tides', 'Opt', 'Riddleform', 'Supreme Will', 'Naru Meha, Master Wizard', 'Essence Flux']
[['Simic Signet', "Ludevic's Test Subject", 'Territorial Allosaurus'], ['Foundry of the Consuls', 'Crawling Sensation', 'Thought Courier', 'Kessig Cagebreakers', 'Greenwarden of Murasa'], ['Grazing Gladehart'], ['Filigree Familiar'], ['Scrounging Bandar']]


In [30]:
[c.name for c in current_build_graph.nodes]

['Bounding Krasis',
 'Dissolve',
 "Familiar's Ruse",
 'Skinshifter',
 'Mana Leak',
 "Wizard's Retort",
 'Negate',
 'Pack Guardian',
 "Centaur's Herald",
 'Rattlechains',
 'Supreme Phantom',
 'Higure, the Still Wind',
 'Birds of Paradise',
 'River Hoopoe',
 'Kiri-Onna',
 'Ninja of the Deep Hours',
 'Prey Upon',
 'Harbinger of the Tides',
 'Opt',
 'Riddleform',
 'Supreme Will',
 'Naru Meha, Master Wizard',
 'Essence Flux']

In [21]:
synergy.sorted_centralities(current_build_graph)

[(Card: {'name': 'Rattlechains', 'color_id': 'U', 'types': ['creature'], 'tags': [('Spirits', 'Payoff'), ('Spirits', 'Enabler'), ('Flash', 'Threat'), ('Ninjutsu', 'Enabler')], 'power_tier': 2},
  0.32201045373989057),
 (Card: {'name': 'Pack Guardian', 'color_id': 'G', 'types': ['creature'], 'tags': [('Land Graveyard', 'Enabler'), ('Spirits', 'Enabler'), ('Flash', 'Threat'), ('Land Extra', 'Payoff')], 'power_tier': 2},
  0.27814527614205503),
 (Card: {'name': 'Supreme Will', 'color_id': 'U', 'types': ['instant'], 'tags': [('Spells', 'Enabler'), ('Flash', 'Interaction')], 'power_tier': 3},
  0.2629951311314047),
 (Card: {'name': 'Naru Meha, Master Wizard', 'color_id': 'U', 'types': ['creature'], 'tags': [('Wizards', 'Payoff'), ('Spells', 'Payoff'), ('Wizards', 'Enabler'), ('Flash', 'Threat')], 'power_tier': 2},
  0.25836940500032196),
 (Card: {'name': "Familiar's Ruse", 'color_id': 'U', 'types': ['instant'], 'tags': [('Flash', 'Interaction')], 'power_tier': 3},
  0.25474268489102514),
 (

In [26]:
c = [(tup[0].name, tup[1]) for tup in synergy.sorted_centralities(on_color_subgraph)]

In [27]:
c[:23]

[('Rattlechains', 0.29630845887279705),
 ("Wizard's Retort", 0.2647268644096714),
 ('Supreme Will', 0.26333962258078975),
 ('Pack Guardian', 0.262981624775553),
 ("Familiar's Ruse", 0.25628391782183085),
 ('Negate', 0.25628391782183074),
 ('Mana Leak', 0.25628391782183074),
 ('Dissolve', 0.2562839178218307),
 ('Naru Meha, Master Wizard', 0.25126430031073516),
 ('River Hoopoe', 0.24357543623002742),
 ('Skinshifter', 0.22424233072290267),
 ('Harbinger of the Tides', 0.22124343975870017),
 ("Ludevic's Test Subject", 0.20976933473291529),
 ('Bounding Krasis', 0.1904362292257906),
 ("Centaur's Herald", 0.19043622922579057),
 ('Higure, the Still Wind', 0.1378616611496144),
 ('Ninja of the Deep Hours', 0.13786166114961435),
 ('Essence Flux', 0.13028498039197567),
 ('Supreme Phantom', 0.1266712425969687),
 ('Birds of Paradise', 0.09574367188970244),
 ('Thought Courier', 0.06886086180014668),
 ('Kiri-Onna', 0.06783511033385614),
 ('Simic Signet', 0.06193757039259032)]

In [28]:
c[23:]

[('Riddleform', 0.05754648631901061),
 ('Crawling Sensation', 0.038417191902993454),
 ('Prey Upon', 0.037862915291868575),
 ('Opt', 0.03786291529186855),
 ('Greenwarden of Murasa', 0.03248633731087179),
 ('Territorial Allosaurus', 0.019333105507124755),
 ('Kessig Cagebreakers', 0.013153231803746994),
 ('Foundry of the Consuls', 0.004710285215906307),
 ('Scrounging Bandar', 3.490632663737966e-17),
 ('Filigree Familiar', -2.3375945112195336e-17),
 ('Grazing Gladehart', -5.328791682561758e-17)]